In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Cookstove_Plan15_for_sharing/data/project_recreation_merged.xlsx')
VERs_all=pd.read_excel('/content/drive/MyDrive/Cookstove_Plan15_for_sharing/results/VERS_merged.xlsx')

## SY added
df['protocol_type'] = df['protocol_type'].replace('CDM AMS-I-E','CDM AMS I E')
df['protocol_type'] = df['protocol_type'].replace('CDM AMS II.G','CDM AMS II G')
##

In [12]:
unique_stoves = df['stove'].unique()
for stove in unique_stoves[1:]:
    var_name = f"{stove.lower()}_VERs"
    globals()[var_name] = df.loc[df['stove'] == stove, 'verified_credits_per_stove'].sum()
    globals()[f"{stove}_N"] = 0
    globals()[f"{stove}_VERs_agw"] = []
    globals()[f"{stove}_ratio_analysis_project_mean_compilation"] = []
    globals()[f"{stove}_ratio_project_analysis_mean_compilation"] = []
    globals()[f"{stove}_ratio_analysis_project_lower_compilation"] = []
    globals()[f"{stove}_ratio_analysis_project_upper_compilation"] = []
    globals()[f"{stove}_ratio_project_analysis_lower_compilation"] = []
    globals()[f"{stove}_ratio_project_analysis_upper_compilation"] = []

# create variables for each country
unique_countries = df['country'].unique()
for country in unique_countries[1:]:
    var_name = f"{country.lower()}_VERs"
    globals()[var_name] = df.loc[df['country'] == country, 'verified_credits_per_stove'].sum()
    # lower the country name
    var_base_name = country.lower().replace(" ", "").replace("-", "").replace("'", "").replace(" ", "")  # lower the country name
    globals()[f"{var_base_name}_VERs_agw"] = []  # 'country_VERs_agw' list
    globals()[f"{var_base_name}_N"] = 0 # 'country_N'
    globals()[f"{var_base_name}_ratio_analysis_project_mean_compilation"] = [] # 'country_ratio_analysis_project_mean_compilation' list
    globals()[f"{var_base_name}_ratio_project_analysis_mean_compilation"] = [] # 'country_ratio_project_analysis_mean_compilation' list
    globals()[f"{var_base_name}_ratio_analysis_project_lower_compilation"] = [] # 'country_ratio_analysis_project_lower_compilation' list
    globals()[f"{var_base_name}_ratio_analysis_project_upper_compilation"] = []   # 'country_ratio_analysis_project_upper_compilation' list
    globals()[f"{var_base_name}_ratio_project_analysis_lower_compilation"] = [] # 'country_ratio_project_analysis_lower_compilation' list
    globals()[f"{var_base_name}_ratio_project_analysis_upper_compilation"] = [] #  'country_ratio_project_analysis_upper_compilation' list

# create variables for each type
unique_types = df['type'].unique()

for type1 in unique_types[1:]:
    globals()[f"{type1}_N"] = 0
    globals()[f"{type1}_VERs_agw"] = []
    globals()[f"{type1}_ratio_analysis_project_mean_compilation"] = []
    globals()[f"{type1}_ratio_project_analysis_mean_compilation"] = []
    globals()[f"{type1}_ratio_analysis_project_lower_compilation"] = []
    globals()[f"{type1}_ratio_analysis_project_upper_compilation"] = []
    globals()[f"{type1}_ratio_project_analysis_lower_compilation"] = []
    globals()[f"{type1}_ratio_project_analysis_upper_compilation"] = []

# create lists for each protocol_type
unique_protocol_types = df['protocol_type'].unique()

for protocol_type in unique_protocol_types[1:]:
  var_protocol_type = protocol_type.replace(" ", "_").replace("_AMS", "")
  globals()[f"{var_protocol_type}_VERs_agw"] = list()
  globals()[f"{var_protocol_type}_ratio_analysis_project_mean_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_project_analysis_mean_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_analysis_project_lower_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_analysis_project_upper_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_project_analysis_lower_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_project_analysis_upper_compilation"] = []

In [13]:
ALL_N = 0
all_VERS_agw = []
ratio_analysis_project_mean_compilation=[]
ratio_project_analysis_mean_compilation=[]
ratio_analysis_project_lower_compilation=[]
ratio_analysis_project_upper_compilation=[]
ratio_project_analysis_lower_compilation=[]
ratio_project_analysis_upper_compilation=[]

# For protocol wide over-estimation
GS_Meter_N=0
GS_N=0
CDM_II_G_N=0
CDM_I_E_N=0

In [14]:
#import relevant packages
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from scipy import stats as stats
from scipy.stats import triang

import seaborn as sns

#triangular distributions

np.random.seed(4)
N = 10000                # random numbers to generate


# Adoption, Usage, and Leakage

  #Adoption
min_adoption=.4
mean_adoption=.58
max_adoption= .92
n_adoption=np.random.triangular(left=min_adoption, mode=mean_adoption, right=max_adoption, size=(N))

req_adoption=np.random.uniform(0,mean_adoption,N)
#Usage
min_usage=0.16
mean_usage=0.52
max_usage=0.85
n_usage=np.random.triangular(left=min_usage, mode=mean_usage, right=max_usage, size=(N))
req_usage=np.random.uniform(0,mean_usage,N)

#Stacking
min_stacking=.193
mean_stacking=.673
max_stacking=.99
n_stacking=np.random.triangular(left=min_stacking, mode=mean_stacking, right=max_stacking, size=(N))
req_stacking=np.random.uniform(0,mean_stacking,N)




# Renewable GWP from Whitman & Lehman

gwp_ch4_renewable=28 #tCO2e/tonne CH4 i.e., instead of typically 25

# Whitman & lehmann 2011 also cite a value for CO
# but we do not include CO in our analysis

gwp_ch4_agw=30
gwp_n2O_agw=298

eff_trad_firewood=0.14
eff_improv_firewood=.21
eff_trad_charcoal=0.2
eff_improv_charcoal=.22
eff_kerosene=.47
eff_LPG=.5
eff_ethanol=.47
eff_pellets=.5 #only two pellet projects is the mimi motto

ncv_firewood_agw=.016
ncv_charcoal_agw=.027
ncv_kerosene_agw=.043
ncv_LPG_agw=.045
ncv_ethanol_agw=.027
ncv_pellets_agw=.017

kg_per_liter_ethanol_agw=0.783 #kg/liter

ef_combustion_co2_firewood=88.71 #tCO2/TJ
ef_combustion_ch4_firewood=0.54 #tCH4/TJ
ef_combustion_n2O_firewood=0 #tN2O/TJ

ef_combustion_co2_charcoal=80.86 #tCO2/TJ
ef_combustion_ch4_charcoal=0.29 #tCH4/TJ
ef_combustion_n2O_charcoal=0 #tN2O/TJ


ef_total_not_combustion_co2_charcoal=140.09 #tCO2/TJ
ef_total_not_combustion_ch4_charcoal=2.19 #tCH4/TJ
ef_total_not_combustion_n2O_charcoal=0.01#tN2O/TJ

ef_total_co2_kerosene=80.14 #tCO2/TJ
ef_total_ch4_kerosene=.11 #tCH4/TJ
ef_total_n2O_kerosene=0 #tN2O/TJ

ef_total_co2_LPG=82.8#tCO2/TJ
ef_total_ch4_LPG=.12 #tCH4/TJ
ef_total_n2O_LPG=0 #tN2O/TJ

ef_total_co2_ethanol=60.45#tCO2/TJ
ef_total_ch4_ethanol=0.61 #tCH4/TJ
ef_total_n2O_ethanol=0.05 #tN2O/TJ

ef_total_co2_pellets=68.44#tCO2/TJ
ef_total_ch4_pellets=0.08#tCH4/TJ
ef_total_n2O_pellets=0 #tN2O/TJ

ef_total_zeroed_co2_ethanol=-9.78#tCO2/TJ
ef_total_zeroed_ch4_ethanol=0.61 #tCH4/TJ
ef_total_zeored_n2O_ethanol=0.05 #tN2O/TJ

ef_total_zeored_co2_pellets=0 #tCO2/TJ
ef_total_zeored_ch4_pellets=0.08 #tCH4/TJ
ef_total_zeored_n2O_pellets=0 #tN2O/TJ

In [15]:
i=1 # The row you want to evaluate
while i<len(df):        ## SY updated
    # first row is units
    VERs=VERs_all.iloc[i,1]
    protocol_type=df['protocol_type'][i]

    protocol_id=df['protocol_id'][i]

    monitoring_period=df['monitoring_period'][i]

    verified_credits=df['verified_credits'][i]

    num_countries=df['num_countries'][i]

    prov_project=df['prov_project'][i]

    fNRB_project=df['fNRB_project'][i]


    mofus_fNRB_bailis=df['mofus_fNRB_bailis'][i]

    n_devices_domestic=df['n_devices_domestic'][i]

    n_devices_commercial=df['n_devices_commercial'][i]

    project_days=df['project_days'][i]

    project_days_commercial=df['project_days_commercial'][i]

    av_hhsize=df['av_hhsize'][i]


    stove_year_cohort=df['stove_year_cohort'][i]

    # Baseline Fuel % What are the baseline fuels in this project location? [put 0% if no use in this location]
    prc_firewood_baseline=df['prc_firewood_baseline'][i]

    prc_charcoal_baseline=df['prc_charcoal_baseline'][i]

    prc_kerosene_baseline=df['prc_kerosene_baseline'][i]

    prc_lpg_baseline=df['prc_lpg_baseline'][i]

    prc_ethanol_baseline=df['prc_ethanol_baseline'][i]

    prc_electric_baseline=df['prc_electric_baseline'][i]

    prc_pellets_baseline=df['prc_pellets_baseline'][i]





    # Baseline Fuel % What are the baseline commercial fuels in this project location? [put 0% if no use in this location]
    prc_firewood_baseline_commercial=df['prc_firewood_baseline_commercial'][i]

    prc_charcoal_baseline_commercial=df['prc_charcoal_baseline_commercial'][i]

    prc_kerosene_baseline_commercial=df['prc_kerosene_baseline_commercial'][i]

    prc_lpg_baseline_commercial=df['prc_lpg_baseline_commercial'][i]

    prc_ethanol_baseline_commercial=df['prc_ethanol_baseline_commercial'][i]

    prc_electric_baseline_commercial=df['prc_electric_baseline_commercial'][i]

    prc_pellets_baseline_commercial=df['prc_pellets_baseline_commercial'][i]






    # Baseline Consumption: What is the baseline fuel consumption of each baseline fuel in this project location [put 0 if no use in this location]
    kg_firewood_baseline=df['kg_firewood_baseline'][i]

    kg_charcoal_baseline=df['kg_charcoal_baseline'][i]

    kg_kerosene_baseline=df['kg_kerosene_baseline'][i]

    kg_lpg_baseline=df['kg_lpg_baseline'][i]

    l_ethanol_baseline=df['l_ethanol_baseline'][i]

    kg_per_liter_ethanol_baseline=df['kg_per_liter_ethanol_baseline'][i]

    kWh_electric_baseline=df['kWh_electric_baseline'][i]

    kg_pellets_baseline=df['kg_pellets_baseline'][i]



    kg_firewood_baseline_commercial=df['kg_firewood_baseline_commercial'][i]

    kg_charcoal_baseline_commercial=df['kg_charcoal_baseline_commercial'][i]

    kg_kerosene_baseline_commercial=df['kg_kerosene_baseline_commercial'][i]

    kg_lpg_baseline_commercial=df['kg_lpg_baseline_commercial'][i]

    l_ethanol_baseline_commercial=df['l_ethanol_baseline_commercial'][i]

    kWh_electric_baseline_commercial=df['kWh_electric_baseline_commercial'][i]

    kg_pellets_baseline_commercial=df['kg_pellets_baseline_commercial'][i]


    # Stove Efficiencies at Baseline


    eff_firewood_baseline=df['eff_firewood_baseline'][i]

    eff_charcoal_baseline=df['eff_charcoal_baseline'][i]

    eff_kerosene_baseline=df['eff_kerosene_baseline'][i]

    eff_LPG_baseline=df['eff_LPG_baseline'][i]

    eff_ethanol_baseline=df['eff_ethanol_baseline'][i]

    eff_electric_baseline=df['eff_electric_baseline'][i]

    eff_pellets_baseline=df['eff_pellets_baseline'][i]


    # Commercial Baseline Stove Efficiences

    eff_firewood_baseline_commercial=df['eff_firewood_baseline_commercial'][i]

    eff_charcoal_baseline_commercial=df['eff_charcoal_baseline_commercial'][i]

    eff_kerosene_baseline_commercial=df['eff_kerosene_baseline_commercial'][i]

    eff_LPG_baseline_commercial=df['eff_LPG_baseline_commercial'][i]

    eff_ethanol_baseline_commercial=df['eff_ethanol_baseline_commercial'][i]

    eff_electric_baseline_commercial=df['eff_electric_baseline_commercial'][i]

    eff_pellet_baseline_commercial=df['eff_pellet_baseline_commercial'][i]


    # Caloric Values: What are the assumed calorific values  of the baseline fuels? (TJ/ton) [put 0 if not in use in this location]

    ncv_firewood_baseline=df['ncv_firewood_baseline'][i]

    ncv_charcoal_baseline=df['ncv_charcoal_baseline'][i]

    ncv_kerosene_baseline=df['ncv_kerosene_baseline'][i]

    ncv_LPG_baseline=df['ncv_LPG_baseline'][i]

    ncv_ethanol_baseline=df['ncv_ethanol_baseline'][i]

    ncv_pellets_baseline=df['ncv_pellets_baseline'][i]





    # GWP: What are the assumed global warming potentials of CH4 & N20? If not included put 0 or leave blank
    gwp_ch4= 25

    gwp_n2O=298


    # CO2 Emission Factors: What is the assumed carbon dioxide emission factor of the baseline fuels?
    #[if CDM write in the assumed EF of the projected fossil fuel]
    #[ put 0 if you do not account for CO2 EF at Baseline or this fuel is not applicable in your location]
    co2_ef_firewood=df['co2_ef_firewood'][i]
    co2_ef_charcoal=df['co2_ef_charcoal'][i]

    co2_ef_kerosene=df['co2_ef_kerosene'][i]

    co2_ef_LPG=df['co2_ef_LPG'][i]

    co2_ef_ethanol=df['co2_ef_ethanol'][i]

    co2_ef_electric=df['co2_ef_electric'][i]

    co2_ef_pellets=df['co2_ef_pellets'][i]


    #Non-CO2 Baseline Emission Factors

    nonco2_ef_firewood=df['nonco2_ef_firewood'][i]
    nonco2_ef_charcoal=df['nonco2_ef_charcoal'][i]

    nonco2_ef_kerosene=df['nonco2_ef_kerosene'][i]

    nonco2_ef_LPG=df['nonco2_ef_LPG'][i]

    nonco2_ef_ethanol=df['nonco2_ef_ethanol'][i]

    nonco2_ef_electric=df['nonco2_ef_electric'][i]

    nonco2_ef_pellets=df['nonco2_ef_pellets'][i]


    # Emissions from Stove Production

    stoveproduction_ef_co2_firewood_baseline= df['stoveproduction_ef_co2_firewood_baseline'][i]
    stoveproduction_ef_ch4_firewood_baseline= df['stoveproduction_ef_ch4_firewood_baseline'][i]
    stoveproduction_ef_n2O_firewood_baseline= df['stoveproduction_ef_n2O_firewood_baseline'][i]


    stoveproduction_ef_co2_charcoal_baseline=df['stoveproduction_ef_co2_charcoal_baseline'][i]
    stoveproduction_ef_ch4_charcoal_baseline=df['stoveproduction_ef_ch4_charcoal_baseline'][i]
    stoveproduction_ef_n2O_charcoal_baseline=df['stoveproduction_ef_n2O_charcoal_baseline'][i]


    stoveproduction_ef_co2_kerosene_baseline=df['stoveproduction_ef_co2_kerosene_baseline'][i]
    stoveproduction_ef_ch4_kerosene_baseline=df['stoveproduction_ef_ch4_kerosene_baseline'][i]
    stoveproduction_ef_n2O_kerosene_baseline=df['stoveproduction_ef_n2O_kerosene_baseline'][i]


    stoveproduction_ef_co2_LPG_baseline=df['stoveproduction_ef_co2_LPG_baseline'][i]
    stoveproduction_ef_ch4_LPG_baseline=df['stoveproduction_ef_ch4_LPG_baseline'][i]
    stoveproduction_ef_n2O_LPG_baseline=df['stoveproduction_ef_n2O_LPG_baseline'][i]


    stoveproduction_ef_co2_ethanol_baseline=df['stoveproduction_ef_co2_ethanol_baseline'][i]
    stoveproduction_ef_ch4_ethanol_baseline=df['stoveproduction_ef_ch4_ethanol_baseline'][i]
    stoveproduction_ef_n2O_ethanol_baseline=df['stoveproduction_ef_n2O_ethanol_baseline'][i]


    stoveproduction_ef_co2_electric_baseline=df['stoveproduction_ef_co2_electric_baseline'][i]
    stoveproduction_ef_ch4_electric_baseline=df['stoveproduction_ef_ch4_electric_baseline'][i]
    stoveproduction_ef_n2O_electric_baseline=df['stoveproduction_ef_n2O_electric_baseline'][i]


    stoveproduction_ef_co2_pellets_baseline=df['stoveproduction_ef_co2_pellets_baseline'][i]
    stoveproduction_ef_ch4_pellets_baseline=df['stoveproduction_ef_ch4_pellets_baseline'][i]
    stoveproduction_ef_n2O_pellets_baseline=df['stoveproduction_ef_n2O_pellets_baseline'][i]


    feedstockproduction_ef_co2_firewood_baseline=df['feedstockproduction_ef_co2_firewood_baseline'][i]
    feedstockproduction_ef_ch4_firewood_baseline=df['feedstockproduction_ef_ch4_firewood_baseline'][i]
    feedstockproduction_ef_n2O_firewood_baseline=df['feedstockproduction_ef_n2O_firewood_baseline'][i]


    feedstockproduction_ef_co2_charcoal_baseline=df['feedstockproduction_ef_co2_charcoal_baseline'][i]
    feedstockproduction_ef_ch4_charcoal_baseline=df['stoveproduction_ef_ch4_charcoal_baseline'][i]
    feedstockproduction_ef_n2O_charcoal_baseline=df['feedstockproduction_ef_n2O_charcoal_baseline'][i]


    feedstockproduction_ef_co2_kerosene_baseline=df['feedstockproduction_ef_co2_kerosene_baseline'][i]
    feedstockproduction_ef_ch4_kerosene_baseline=df['feedstockproduction_ef_ch4_kerosene_baseline'][i]
    feedstockproduction_ef_n2O_kerosene_baseline=df['feedstockproduction_ef_n2O_kerosene_baseline'][i]


    feedstockproduction_ef_co2_LPG_baseline=df['feedstockproduction_ef_co2_LPG_baseline'][i]
    feedstockproduction_ef_ch4_LPG_baseline=df['feedstockproduction_ef_ch4_LPG_baseline'][i]
    feedstockproduction_ef_n2O_LPG_baseline=df['feedstockproduction_ef_n2O_LPG_baseline'][i]


    feedstockproduction_ef_co2_ethanol_baseline=df['feedstockproduction_ef_co2_firewood_baseline'][i]
    feedstockproduction_ef_ch4_ethanol_baseline=df['feedstockproduction_ef_ch4_firewood_baseline'][i]
    feedstockproduction_ef_n2O_ethanol_baseline=df['feedstockproduction_ef_n2O_firewood_baseline'][i]


    feedstockproduction_ef_co2_electric_baseline=df['feedstockproduction_ef_co2_firewood_baseline'][i]
    feedstockproduction_ef_ch4_electric_baseline=df['feedstockproduction_ef_ch4_firewood_baseline'][i]
    feedstockproduction_ef_n2O_electric_baseline=df['feedstockproduction_ef_n2O_firewood_baseline'][i]


    feedstockproduction_ef_co2_pellets_baseline=df['feedstockproduction_ef_co2_firewood_baseline'][i]
    feedstockproduction_ef_ch4_pellets_baseline=df['feedstockproduction_ef_ch4_firewood_baseline'][i]
    feedstockproduction_ef_n2O_pellets_baseline=df['feedstockproduction_ef_n2O_firewood_baseline'][i]


    distribution_ef_co2_firewood_baseline=df['distribution_ef_co2_firewood_baseline'][i]
    distribution_ef_ch4_firewood_baseline=df['distribution_ef_ch4_firewood_baseline'][i]
    distribution_ef_n2O_firewood_baseline=df['distribution_ef_n2O_firewood_baseline'][i]


    distribution_ef_co2_charcoal_baseline=df['distribution_ef_co2_charcoal_baseline'][i]
    distribution_ef_ch4_charcoal_baseline=df['distribution_ef_ch4_charcoal_baseline'][i]
    distribution_ef_n2O_charcoal_baseline=df['distribution_ef_n2O_charcoal_baseline'][i]


    distribution_ef_co2_kerosene_baseline=df['distribution_ef_co2_kerosene_baseline'][i]
    distribution_ef_ch4_kerosene_baseline=df['distribution_ef_ch4_kerosene_baseline'][i]
    distribution_ef_n2O_kerosene_baseline=df['distribution_ef_n2O_kerosene_baseline'][i]


    distribution_ef_co2_LPG_baseline=df['distribution_ef_co2_LPG_baseline'][i]
    distribution_ef_ch4_LPG_baseline=df['distribution_ef_ch4_LPG_baseline'][i]
    distribution_ef_n2O_LPG_baseline=df['distribution_ef_n2O_LPG_baseline'][i]


    distribution_ef_co2_ethanol_baseline=df['distribution_ef_co2_ethanol_baseline'][i]
    distribution_ef_ch4_ethanol_baseline=df['distribution_ef_ch4_ethanol_baseline'][i]
    distribution_ef_n2O_ethanol_baseline=df['distribution_ef_n2O_ethanol_baseline'][i]


    distribution_ef_co2_electric_baseline=df['distribution_ef_co2_electric_baseline'][i]
    distribution_ef_ch4_electric_baseline=df['distribution_ef_ch4_electric_baseline'][i]
    distribution_ef_n2O_electric_baseline=df['distribution_ef_n2O_electric_baseline'][i]


    distribution_ef_co2_pellets_baseline=df['distribution_ef_co2_pellets_baseline'][i]
    distribution_ef_ch4_pellets_baseline=df['distribution_ef_ch4_pellets_baseline'][i]
    distribution_ef_n2O_pellets_baseline=df['distribution_ef_n2O_pellets_baseline'][i]


    processing_ef_co2_firewood_baseline=df['processing_ef_co2_firewood_baseline'][i]
    processing_ef_ch4_firewood_baseline=df['processing_ef_ch4_firewood_baseline'][i]
    processing_ef_n2O_firewood_baseline=df['processing_ef_n2O_firewood_baseline'][i]


    processing_ef_co2_charcoal_baseline=df['processing_ef_co2_firewood_baseline'][i]
    processing_ef_ch4_charcoal_baseline=df['processing_ef_co2_firewood_baseline'][i]
    processing_ef_n2O_charcoal_baseline=df['processing_ef_co2_firewood_baseline'][i]


    processing_ef_co2_kerosene_baseline=df['processing_ef_co2_kerosene_baseline'][i]
    processing_ef_ch4_kerosene_baseline=df['processing_ef_ch4_kerosene_baseline'][i]
    processing_ef_n2O_kerosene_baseline=df['processing_ef_n2O_kerosene_baseline'][i]


    processing_ef_co2_LPG_baseline=df['processing_ef_co2_LPG_baseline'][i]
    processing_ef_ch4_LPG_baseline=df['processing_ef_ch4_LPG_baseline'][i]
    processing_ef_n2O_LPG_baseline=df['processing_ef_n2O_LPG_baseline'][i]


    processing_ef_co2_ethanol_baseline=df['processing_ef_co2_ethanol_baseline'][i]
    processing_ef_ch4_ethanol_baseline=df['processing_ef_ch4_ethanol_baseline'][i]
    processing_ef_n2O_ethanol_baseline=df['processing_ef_n2O_ethanol_baseline'][i]


    processing_ef_co2_electric_baseline=df['processing_ef_co2_electric_baseline'][i]
    processing_ef_ch4_electric_baseline=df['processing_ef_ch4_electric_baseline'][i]
    processing_ef_n2O_electric_baseline=df['processing_ef_n2O_electric_baseline'][i]


    processing_ef_co2_pellets_baseline=df['processing_ef_co2_firewood_baseline'][i]
    processing_ef_ch4_pellets_baseline=df['processing_ef_ch4_firewood_baseline'][i]
    processing_ef_n2O_pellets_baseline=df['processing_ef_n2O_firewood_baseline'][i]

    # Project Fuel % What are the project domestic fuels in this project location? [put 0% if no use in this location]
    prc_firewood_project=df['prc_firewood_project'][i]

    prc_charcoal_project=df['prc_charcoal_project'][i]

    prc_kerosene_project=df['prc_kerosene_project'][i]

    prc_lpg_project=df['prc_lpg_project'][i]

    prc_ethanol_project=df['prc_ethanol_project'][i]

    prc_electric_project=df['prc_electric_project'][i]

    prc_pellets_project=df['prc_pellets_project'][i]



    # Project Fuel % What are the project commercial fuels in this project location? [put 0% if no use in this location]
    prc_firewood_project_commercial=df['prc_firewood_project_commercial'][i]

    prc_charcoal_project_commercial=df['prc_charcoal_project_commercial'][i]

    prc_kerosene_project_commercial=df['prc_kerosene_project_commercial'][i]

    prc_lpg_project_commercial=df['prc_lpg_project_commercial'][i]

    prc_ethanol_project_commercial=df['prc_ethanol_project_commercial'][i]

    prc_electric_project_commercial=df['prc_electric_project_commercial'][i]

    prc_pellets_project_commercial=df['prc_pellets_project_commercial'][i]



    # Project Fuel Consumption

    kg_firewood_project=df['kg_firewood_project'][i]

    kg_charcoal_project=df['kg_charcoal_project'][i]

    kg_kerosene_project=df['kg_kerosene_project'][i]

    kg_lpg_project=df['kg_lpg_project'][i]

    l_ethanol_project=df['l_ethanol_project'][i]


    kg_per_liter_ethanol_project=df['kg_per_liter_ethanol_project'][i]

    kWh_electric_project=df['kWh_electric_project'][i]

    kg_pellets_project=df['kg_pellets_project'][i]


    # Project Insitutional Consumption


    kg_firewood_project_commercial=df['kg_firewood_project_commercial'][i]

    kg_charcoal_project_commercial=df['kg_charcoal_project_commercial'][i]

    kg_kerosene_project_commercial=df['kg_kerosene_project_commercial'][i]

    kg_lpg_project_commercial=df['kg_lpg_project_commercial'][i]

    l_ethanol_project_commercial=df['l_ethanol_project_commercial'][i]

    kWh_electric_project_commercial=df['kWh_electric_project_commercial'][i]

    kg_pellets_project_commercial=df['kg_pellets_project_commercial'][i]




    # Project Stove Efficiencies

    eff_firewood_project=df['eff_firewood_project'][i]

    eff_charcoal_project=df['eff_charcoal_project'][i]

    eff_kerosene_project=df['eff_kerosene_project'][i]

    eff_LPG_project=df['eff_LPG_project'][i]

    eff_ethanol_project=df['eff_ethanol_project'][i]

    eff_electric_project=df['eff_electric_project'][i]

    eff_pellet_project=df['eff_pellet_project'][i]


    # Project Instituitonal Stove Efficiencies

    eff_firewood_project_commercial=df['eff_firewood_project_commercial'][i]

    eff_charcoal_project_commercial=df['eff_charcoal_project_commercial'][i]

    eff_kerosene_project_commercial=df['eff_kerosene_project_commercial'][i]

    eff_LPG_project_commercial=df['eff_LPG_project_commercial'][i]

    eff_ethanol_project_commercial=df['eff_ethanol_project_commercial'][i]

    eff_electric_project_commercial=df['eff_electric_project_commercial'][i]

    eff_pellet_project_commercial=df['eff_pellet_project_commercial'][i]


    #Assumed Caloric Values in the Project Scenario

    ncv_firewood_project=df['ncv_firewood_project'][i]

    ncv_charcoal_project=df['ncv_charcoal_project'][i]

    ncv_kerosene_project=df['ncv_kerosene_project'][i]

    ncv_LPG_project=df['ncv_LPG_project'][i]

    ncv_ethanol_project=df['ncv_ethanol_project'][i]

    ncv_pellets_project=df['ncv_pellets_project'][i]





    # Assumed GWP potentials of non-Co2 gases in the project
    gwp_ch4_project=25

    gwp_n2O_project=298


    # CO2 Emission Factors in the Project

    co2_ef_firewood_project=df['co2_ef_firewood_project'][i]

    co2_ef_charcoal_project=df['co2_ef_charcoal_project'][i]

    co2_ef_kerosene_project=df['co2_ef_kerosene_project'][i]

    co2_ef_LPG_project=df['co2_ef_LPG_project'][i]

    co2_ef_ethanol_project=df['co2_ef_ethanol_project'][i]

    co2_ef_electric_project=df['co2_ef_electric_project'][i]

    co2_ef_pellets_project=df['co2_ef_pellets_project'][i]


    # Non-CO2 Emission Factors in the Project

    nonco2_ef_firewood_project=df['nonco2_ef_firewood_project'][i]

    nonco2_ef_charcoal_project=df['nonco2_ef_charcoal_project'][i]

    nonco2_ef_kerosene_project=df['nonco2_ef_kerosene_project'][i]

    nonco2_ef_LPG_project=df['nonco2_ef_LPG_project'][i]

    nonco2_ef_ethanol_project=df['nonco2_ef_ethanol_project'][i]

    nonco2_ef_electric_project=df['nonco2_ef_electric_project'][i]

    nonco2_ef_pellets_project=df['nonco2_ef_pellets_project'][i]

    # Stove Production Emission Factors


    stoveproduction_ef_co2_firewood_project=df['stoveproduction_ef_co2_firewood_project'][i]
    stoveproduction_ef_ch4_firewood_project=df['stoveproduction_ef_ch4_firewood_project'][i]
    stoveproduction_ef_n2O_firewood_project=df['stoveproduction_ef_n2O_firewood_project'][i]


    stoveproduction_ef_co2_charcoal_project=df['stoveproduction_ef_co2_charcoal_project'][i]
    stoveproduction_ef_ch4_charcoal_project=df['stoveproduction_ef_ch4_charcoal_project'][i]
    stoveproduction_ef_n2O_charcoal_project=df['stoveproduction_ef_n2O_charcoal_project'][i]


    stoveproduction_ef_co2_kerosene_project=df['stoveproduction_ef_co2_kerosene_project'][i]
    stoveproduction_ef_ch4_kerosene_project=df['stoveproduction_ef_ch4_kerosene_project'][i]
    stoveproduction_ef_n2O_kerosene_project=df['stoveproduction_ef_n2O_kerosene_project'][i]


    stoveproduction_ef_co2_LPG_project=df['stoveproduction_ef_co2_LPG_project'][i]
    stoveproduction_ef_ch4_LPG_project=df['stoveproduction_ef_ch4_LPG_project'][i]
    stoveproduction_ef_n2O_LPG_project=df['stoveproduction_ef_n2O_LPG_project'][i]


    stoveproduction_ef_co2_ethanol_project=df['stoveproduction_ef_co2_ethanol_project'][i]
    stoveproduction_ef_ch4_ethanol_project=df['stoveproduction_ef_ch4_ethanol_project'][i]
    stoveproduction_ef_n2O_ethanol_project=df['stoveproduction_ef_n2O_ethanol_project'][i]


    stoveproduction_ef_co2_electric_project=df['stoveproduction_ef_co2_electric_project'][i]
    stoveproduction_ef_ch4_electric_project=df['stoveproduction_ef_ch4_electric_project'][i]
    stoveproduction_ef_n2O_electric_project=df['stoveproduction_ef_n2O_electric_project'][i]


    stoveproduction_ef_co2_pellets_project=df['stoveproduction_ef_co2_pellets_project'][i]
    stoveproduction_ef_ch4_pellets_project=df['stoveproduction_ef_ch4_pellets_project'][i]
    stoveproduction_ef_n2O_pellets_project=df['stoveproduction_ef_n2O_pellets_project'][i]

    # Project Feedstock Emission Factors

    feedstockproduction_ef_co2_firewood_project=df['feedstockproduction_ef_co2_firewood_project'][i]
    feedstockproduction_ef_ch4_firewood_project=df['feedstockproduction_ef_ch4_firewood_project'][i]
    feedstockproduction_ef_n2O_firewood_project=df['feedstockproduction_ef_n2O_firewood_project'][i]


    feedstockproduction_ef_co2_charcoal_project=df['feedstockproduction_ef_co2_charcoal_project'][i]
    feedstockproduction_ef_ch4_charcoal_project=df['feedstockproduction_ef_ch4_charcoal_project'][i]
    feedstockproduction_ef_n2O_charcoal_project=df['feedstockproduction_ef_n2O_charcoal_project'][i]


    feedstockproduction_ef_co2_kerosene_project=df['feedstockproduction_ef_co2_kerosene_project'][i]
    feedstockproduction_ef_ch4_kerosene_project=df['feedstockproduction_ef_ch4_kerosene_project'][i]
    feedstockproduction_ef_n2O_kerosene_project=df['feedstockproduction_ef_n2O_kerosene_project'][i]


    feedstockproduction_ef_co2_LPG_project=df['feedstockproduction_ef_co2_LPG_project'][i]
    feedstockproduction_ef_ch4_LPG_project=df['feedstockproduction_ef_ch4_LPG_project'][i]
    feedstockproduction_ef_n2O_LPG_project=df['feedstockproduction_ef_n2O_LPG_project'][i]


    feedstockproduction_ef_co2_ethanol_project=df['feedstockproduction_ef_co2_ethanol_project'][i]
    feedstockproduction_ef_ch4_ethanol_project=df['feedstockproduction_ef_ch4_ethanol_project'][i]
    feedstockproduction_ef_n2O_ethanol_project=df['feedstockproduction_ef_n2O_ethanol_project'][i]


    feedstockproduction_ef_co2_electric_project=df['feedstockproduction_ef_co2_electric_project'][i]
    feedstockproduction_ef_ch4_electric_project=df['feedstockproduction_ef_ch4_electric_project'][i]
    feedstockproduction_ef_n2O_electric_project=df['feedstockproduction_ef_n2O_electric_project'][i]


    feedstockproduction_ef_co2_pellets_project=df['feedstockproduction_ef_co2_pellets_project'][i]
    feedstockproduction_ef_ch4_pellets_project=df['feedstockproduction_ef_ch4_pellets_project'][i]
    feedstockproduction_ef_n2O_pellets_project=df['feedstockproduction_ef_n2O_pellets_project'][i]


    # Project Distribution Emission Factors
    distribution_ef_co2_firewood_project=df['distribution_ef_co2_firewood_project'][i]
    distribution_ef_ch4_firewood_project=df['distribution_ef_ch4_firewood_project'][i]
    distribution_ef_n2O_firewood_project=df['distribution_ef_n2O_firewood_project'][i]


    distribution_ef_co2_charcoal_project=df['distribution_ef_co2_charcoal_project'][i]
    distribution_ef_ch4_charcoal_project=df['distribution_ef_ch4_charcoal_project'][i]
    distribution_ef_n2O_charcoal_project=df['distribution_ef_n2O_charcoal_project'][i]


    distribution_ef_co2_kerosene_project=df['distribution_ef_co2_kerosene_project'][i]
    distribution_ef_ch4_kerosene_project=df['distribution_ef_ch4_kerosene_project'][i]
    distribution_ef_n2O_kerosene_project=df['distribution_ef_n2O_kerosene_project'][i]


    distribution_ef_co2_LPG_project=df['distribution_ef_co2_LPG_project'][i]
    distribution_ef_ch4_LPG_project=df['distribution_ef_ch4_LPG_project'][i]
    distribution_ef_n2O_LPG_project=df['distribution_ef_n2O_LPG_project'][i]


    distribution_ef_co2_ethanol_project=df['distribution_ef_co2_ethanol_project'][i]
    distribution_ef_ch4_ethanol_project=df['distribution_ef_ch4_ethanol_project'][i]
    distribution_ef_n2O_ethanol_project=df['distribution_ef_n2O_ethanol_project'][i]


    distribution_ef_co2_electric_project=df['distribution_ef_co2_electric_project'][i]
    distribution_ef_ch4_electric_project=df['distribution_ef_ch4_electric_project'][i]
    distribution_ef_n2O_electric_project=df['distribution_ef_n2O_electric_project'][i]


    distribution_ef_co2_pellets_project=df['distribution_ef_co2_pellets_project'][i]
    distribution_ef_ch4_pellets_project=df['distribution_ef_ch4_pellets_project'][i]
    distribution_ef_n2O_pellets_project=df['distribution_ef_n2O_pellets_project'][i]

    # Project Processing Emission Factors
    processing_ef_co2_firewood_project=df['processing_ef_co2_firewood_project'][i]
    processing_ef_ch4_firewood_project=df['processing_ef_ch4_firewood_project'][i]
    processing_ef_n2O_firewood_project=df['processing_ef_n2O_firewood_project'][i]


    processing_ef_co2_charcoal_project=df['processing_ef_co2_charcoal_project'][i]
    processing_ef_ch4_charcoal_project=df['processing_ef_ch4_charcoal_project'][i]
    processing_ef_n2O_charcoal_project=df['processing_ef_n2O_charcoal_project'][i]


    processing_ef_co2_kerosene_project=df['processing_ef_co2_kerosene_project'][i]
    processing_ef_ch4_kerosene_project=df['processing_ef_ch4_kerosene_project'][i]
    processing_ef_n2O_kerosene_project=df['processing_ef_n2O_kerosene_project'][i]


    processing_ef_co2_LPG_project=df['processing_ef_co2_LPG_project'][i]
    processing_ef_ch4_LPG_project=df['processing_ef_ch4_LPG_project'][i]
    processing_ef_n2O_LPG_project=df['processing_ef_n2O_LPG_project'][i]


    processing_ef_co2_ethanol_project=df['processing_ef_co2_ethanol_project'][i]
    processing_ef_ch4_ethanol_project=df['processing_ef_ch4_ethanol_project'][i]
    processing_ef_n2O_ethanol_project=df['processing_ef_n2O_ethanol_project'][i]


    processing_ef_co2_electric_project=df['processing_ef_co2_electric_project'][i]
    processing_ef_ch4_electric_project=df['processing_ef_ch4_electric_project'][i]
    processing_ef_n2O_electric_project=df['processing_ef_n2O_electric_project'][i]


    processing_ef_co2_pellets_project=df['processing_ef_co2_pellets_project'][i]
    processing_ef_ch4_pellets_project=df['processing_ef_ch4_pellets_project'][i]
    processing_ef_n2O_pellets_project=df['processing_ef_n2O_pellets_project'][i]

    #fNRB


    n_fNRB=fNRB_project

    # Adoption, Usage, and Leakage

    #adoption_definition

    project_adoption=df['project_adoption'][i]


    project_usage=df['project_usage'][i]


    project_adoption_commercial=df['project_adoption_commercial'][i]


    project_usage_commercial=df['project_usage_commercial'][i]

    kpt_adj_usage=np.random.uniform(0,0.53,N)

    kpt_usage=project_usage*(1-kpt_adj_usage)

    project_stacking=df['project_stacking'][i]

    project_stacking_commercial=df['project_stacking_commercial'][i]

    kpt_adj_stacking=(1-0.23)

    kpt_stacking=project_stacking*kpt_adj_stacking

    leakage_binary=df['leakage_binary'][i]



    leakage_default= df['leakage_default'][i] # put 1 if not being multiplied



    leakage_tCO2e=df['leakage_tCO2e'][i]



    more_than_one_stove_adj=df['more_than_one_stove_adj'][i]

    other_tCO2e_reduction=df['other_tCO2e_reduction'][i]

    #rebound=1

    rebound=df['rebound'][i]



    # Baseline Domestic Emissions
    # Sum all baseline domestic emissions from each baseline fuel/technology

    # Baseline Firewood
    be_firewood_gillwiehl=prc_firewood_baseline*(ncv_firewood_baseline*kg_firewood_baseline/1000*((n_fNRB*co2_ef_firewood)
                                        + nonco2_ef_firewood

                                        + feedstockproduction_ef_co2_firewood_baseline
                                        + feedstockproduction_ef_ch4_firewood_baseline*gwp_ch4
                                        + feedstockproduction_ef_n2O_firewood_baseline*gwp_n2O

                                        + distribution_ef_co2_firewood_baseline
                                        + distribution_ef_ch4_firewood_baseline*gwp_ch4
                                        + distribution_ef_n2O_firewood_baseline*gwp_n2O

                                        + processing_ef_co2_firewood_baseline
                                        + processing_ef_ch4_firewood_baseline*gwp_ch4
                                        + processing_ef_n2O_firewood_baseline*gwp_n2O)

                                        + stoveproduction_ef_co2_firewood_baseline
                                        + stoveproduction_ef_ch4_firewood_baseline*gwp_ch4
                                        + stoveproduction_ef_n2O_firewood_baseline*gwp_n2O)


    # Baseline Charcoal

    be_charcoal_gillwiehl=prc_charcoal_baseline*(ncv_charcoal_baseline*kg_charcoal_baseline/1000*((n_fNRB*co2_ef_charcoal)
                                           + nonco2_ef_charcoal

                                           + feedstockproduction_ef_co2_charcoal_baseline
                                           + feedstockproduction_ef_ch4_charcoal_baseline*gwp_ch4
                                           + feedstockproduction_ef_n2O_charcoal_baseline*gwp_n2O

                                           + distribution_ef_co2_charcoal_baseline
                                           + distribution_ef_ch4_charcoal_baseline*gwp_ch4
                                           + distribution_ef_n2O_charcoal_baseline*gwp_n2O

                                           + processing_ef_co2_charcoal_baseline
                                           + processing_ef_ch4_charcoal_baseline*gwp_ch4
                                           + processing_ef_n2O_charcoal_baseline*gwp_n2O)

                                           + stoveproduction_ef_co2_charcoal_baseline
                                           + stoveproduction_ef_ch4_charcoal_baseline*gwp_ch4
                                           + stoveproduction_ef_n2O_charcoal_baseline*gwp_n2O)


    # Baseline LPG

    be_LPG_gillwiehl=prc_lpg_baseline*(ncv_LPG_baseline*kg_lpg_baseline/1000*(co2_ef_LPG +nonco2_ef_LPG

                                           + feedstockproduction_ef_co2_charcoal_baseline
                                           + feedstockproduction_ef_ch4_charcoal_baseline*gwp_ch4
                                           + feedstockproduction_ef_n2O_charcoal_baseline*gwp_n2O

                                           + distribution_ef_co2_charcoal_baseline
                                           + distribution_ef_ch4_charcoal_baseline*gwp_ch4
                                           + distribution_ef_n2O_charcoal_baseline*gwp_n2O

                                           + processing_ef_co2_charcoal_baseline
                                           + processing_ef_ch4_charcoal_baseline*gwp_ch4
                                           + processing_ef_n2O_charcoal_baseline*gwp_n2O)

                                           + stoveproduction_ef_co2_charcoal_baseline
                                           + stoveproduction_ef_ch4_charcoal_baseline*gwp_ch4
                                           + stoveproduction_ef_n2O_charcoal_baseline*gwp_n2O)


    # Baseline kerosene

    be_kerosene_gillwiehl=prc_kerosene_baseline*(ncv_kerosene_baseline*kg_kerosene_baseline/1000*(co2_ef_kerosene +nonco2_ef_kerosene

                                           + feedstockproduction_ef_co2_kerosene_baseline
                                           + feedstockproduction_ef_ch4_kerosene_baseline*gwp_ch4
                                           + feedstockproduction_ef_n2O_kerosene_baseline*gwp_n2O

                                           + distribution_ef_co2_kerosene_baseline
                                           + distribution_ef_ch4_kerosene_baseline*gwp_ch4
                                           + distribution_ef_n2O_kerosene_baseline*gwp_n2O

                                           + processing_ef_co2_kerosene_baseline
                                           + processing_ef_ch4_kerosene_baseline*gwp_ch4
                                           + processing_ef_n2O_kerosene_baseline*gwp_n2O)

                                           + stoveproduction_ef_co2_kerosene_baseline
                                           + stoveproduction_ef_ch4_kerosene_baseline*gwp_ch4
                                           + stoveproduction_ef_n2O_kerosene_baseline*gwp_n2O)


    # Baseline electricity

    be_electric_gillwiehl=prc_electric_baseline*(kWh_electric_baseline/1000*(co2_ef_electric+nonco2_ef_electric

                                           + feedstockproduction_ef_co2_electric_baseline
                                           + feedstockproduction_ef_ch4_electric_baseline*gwp_ch4
                                           + feedstockproduction_ef_n2O_electric_baseline*gwp_n2O

                                           + distribution_ef_co2_electric_baseline
                                           + distribution_ef_ch4_electric_baseline*gwp_ch4
                                           + distribution_ef_n2O_electric_baseline*gwp_n2O

                                           + processing_ef_co2_electric_baseline
                                           + processing_ef_ch4_electric_baseline*gwp_ch4
                                           + processing_ef_n2O_electric_baseline*gwp_n2O)

                                           + stoveproduction_ef_co2_electric_baseline
                                           + stoveproduction_ef_ch4_electric_baseline*gwp_ch4
                                           + stoveproduction_ef_n2O_electric_baseline*gwp_n2O)


    # Baseline ethanol

    be_ethanol_gillwiehl=prc_ethanol_baseline*(ncv_ethanol_baseline*l_ethanol_baseline*kg_per_liter_ethanol_baseline/1000*(co2_ef_ethanol+nonco2_ef_ethanol

                                           + feedstockproduction_ef_co2_ethanol_baseline
                                           + feedstockproduction_ef_ch4_ethanol_baseline*gwp_ch4
                                           + feedstockproduction_ef_n2O_ethanol_baseline*gwp_n2O

                                           + distribution_ef_co2_ethanol_baseline
                                           + distribution_ef_ch4_ethanol_baseline*gwp_ch4
                                           + distribution_ef_n2O_ethanol_baseline*gwp_n2O

                                           + processing_ef_co2_ethanol_baseline
                                           + processing_ef_ch4_ethanol_baseline*gwp_ch4
                                           + processing_ef_n2O_ethanol_baseline*gwp_n2O)

                                           + stoveproduction_ef_co2_ethanol_baseline
                                           + stoveproduction_ef_ch4_ethanol_baseline*gwp_ch4
                                           + stoveproduction_ef_n2O_ethanol_baseline*gwp_n2O)


    # Baseline pellets

    be_pellets_gillwiehl=prc_pellets_baseline*(ncv_pellets_baseline*kg_pellets_baseline/1000*((n_fNRB*co2_ef_pellets)
                                           + nonco2_ef_pellets

                                           + feedstockproduction_ef_co2_pellets_baseline
                                           + feedstockproduction_ef_ch4_pellets_baseline*gwp_ch4
                                           + feedstockproduction_ef_n2O_pellets_baseline*gwp_n2O

                                           + distribution_ef_co2_pellets_baseline
                                           + distribution_ef_ch4_pellets_baseline*gwp_ch4
                                           + distribution_ef_n2O_pellets_baseline*gwp_n2O

                                           + processing_ef_co2_pellets_baseline
                                           + processing_ef_ch4_pellets_baseline*gwp_ch4
                                           + processing_ef_n2O_pellets_baseline*gwp_n2O)

                                           + stoveproduction_ef_co2_pellets_baseline
                                           + stoveproduction_ef_ch4_pellets_baseline*gwp_ch4
                                           + stoveproduction_ef_n2O_pellets_baseline*gwp_n2O)

    be_total_domestic_gillwiehl = be_firewood_gillwiehl + be_charcoal_gillwiehl + be_LPG_gillwiehl+ be_electric_gillwiehl + be_ethanol_gillwiehl + be_pellets_gillwiehl

    be_total_domestic_gillwiehl



    # Sum all baseline commercial emissions from each baseline fuel/technology

    # Baseline Commercial Firewood

    be_firewood_commercial_gillwiehl=prc_firewood_baseline_commercial*(ncv_firewood_baseline*kg_firewood_baseline_commercial/1000*((n_fNRB*co2_ef_firewood)
                                        + nonco2_ef_firewood

                                        + feedstockproduction_ef_co2_firewood_baseline
                                        + feedstockproduction_ef_ch4_firewood_baseline*gwp_ch4
                                        + feedstockproduction_ef_n2O_firewood_baseline*gwp_n2O

                                        + distribution_ef_co2_firewood_baseline
                                        + distribution_ef_ch4_firewood_baseline*gwp_ch4
                                        + distribution_ef_n2O_firewood_baseline*gwp_n2O

                                        + processing_ef_co2_firewood_baseline
                                        + processing_ef_ch4_firewood_baseline*gwp_ch4
                                        + processing_ef_n2O_firewood_baseline*gwp_n2O)

                                        + stoveproduction_ef_co2_firewood_baseline
                                        + stoveproduction_ef_ch4_firewood_baseline*gwp_ch4
                                        + stoveproduction_ef_n2O_firewood_baseline*gwp_n2O)


    # Baseline  Commercial Charcoal

    be_charcoal_commercial_gillwiehl=prc_charcoal_baseline_commercial*(ncv_charcoal_baseline*kg_charcoal_baseline_commercial/1000*((n_fNRB*co2_ef_charcoal)
                                           + nonco2_ef_charcoal

                                           + feedstockproduction_ef_co2_charcoal_baseline
                                           + feedstockproduction_ef_ch4_charcoal_baseline*gwp_ch4
                                           + feedstockproduction_ef_n2O_charcoal_baseline*gwp_n2O

                                           + distribution_ef_co2_charcoal_baseline
                                           + distribution_ef_ch4_charcoal_baseline*gwp_ch4
                                           + distribution_ef_n2O_charcoal_baseline*gwp_n2O

                                           + processing_ef_co2_charcoal_baseline
                                           + processing_ef_ch4_charcoal_baseline*gwp_ch4
                                           + processing_ef_n2O_charcoal_baseline*gwp_n2O)

                                           + stoveproduction_ef_co2_charcoal_baseline
                                           + stoveproduction_ef_ch4_charcoal_baseline*gwp_ch4
                                           + stoveproduction_ef_n2O_charcoal_baseline*gwp_n2O)


    # Baseline LPG

    be_LPG_commercial_gillwiehl=prc_lpg_baseline_commercial*(ncv_LPG_baseline*kg_lpg_baseline_commercial/1000*(co2_ef_LPG +nonco2_ef_LPG

                                           + feedstockproduction_ef_co2_charcoal_baseline
                                           + feedstockproduction_ef_ch4_charcoal_baseline*gwp_ch4
                                           + feedstockproduction_ef_n2O_charcoal_baseline*gwp_n2O

                                           + distribution_ef_co2_charcoal_baseline
                                           + distribution_ef_ch4_charcoal_baseline*gwp_ch4
                                           + distribution_ef_n2O_charcoal_baseline*gwp_n2O

                                           + processing_ef_co2_charcoal_baseline
                                           + processing_ef_ch4_charcoal_baseline*gwp_ch4
                                           + processing_ef_n2O_charcoal_baseline*gwp_n2O)

                                           + stoveproduction_ef_co2_charcoal_baseline
                                           + stoveproduction_ef_ch4_charcoal_baseline*gwp_ch4
                                           + stoveproduction_ef_n2O_charcoal_baseline*gwp_n2O)


    # Baseline  Commercial Kerosene

    be_kerosene_commercial_gillwiehl=prc_kerosene_baseline_commercial*(ncv_kerosene_baseline*kg_kerosene_baseline_commercial/1000*(co2_ef_kerosene +nonco2_ef_kerosene

                                           + feedstockproduction_ef_co2_kerosene_baseline
                                           + feedstockproduction_ef_ch4_kerosene_baseline*gwp_ch4
                                           + feedstockproduction_ef_n2O_kerosene_baseline*gwp_n2O

                                           + distribution_ef_co2_kerosene_baseline
                                           + distribution_ef_ch4_kerosene_baseline*gwp_ch4
                                           + distribution_ef_n2O_kerosene_baseline*gwp_n2O

                                           + processing_ef_co2_kerosene_baseline
                                           + processing_ef_ch4_kerosene_baseline*gwp_ch4
                                           + processing_ef_n2O_kerosene_baseline*gwp_n2O)

                                           + stoveproduction_ef_co2_kerosene_baseline
                                           + stoveproduction_ef_ch4_kerosene_baseline*gwp_ch4
                                           + stoveproduction_ef_n2O_kerosene_baseline*gwp_n2O)


    # Baseline  Commercial Electricity

    be_electric_commercial_gillwiehl=prc_electric_baseline_commercial*(kWh_electric_baseline_commercial/1000*(co2_ef_electric+nonco2_ef_electric

                                           + feedstockproduction_ef_co2_electric_baseline
                                           + feedstockproduction_ef_ch4_electric_baseline*gwp_ch4
                                           + feedstockproduction_ef_n2O_electric_baseline*gwp_n2O

                                           + distribution_ef_co2_electric_baseline
                                           + distribution_ef_ch4_electric_baseline*gwp_ch4
                                           + distribution_ef_n2O_electric_baseline*gwp_n2O

                                           + processing_ef_co2_electric_baseline
                                           + processing_ef_ch4_electric_baseline*gwp_ch4
                                           + processing_ef_n2O_electric_baseline*gwp_n2O)

                                           + stoveproduction_ef_co2_electric_baseline
                                           + stoveproduction_ef_ch4_electric_baseline*gwp_ch4
                                           + stoveproduction_ef_n2O_electric_baseline*gwp_n2O)


    # Baseline  Commercial Ethanol

    be_ethanol_commercial_gillwiehl=prc_ethanol_baseline_commercial*(ncv_ethanol_baseline*l_ethanol_baseline_commercial*kg_per_liter_ethanol_baseline/1000*(co2_ef_ethanol+nonco2_ef_ethanol

                                           + feedstockproduction_ef_co2_ethanol_baseline
                                           + feedstockproduction_ef_ch4_ethanol_baseline*gwp_ch4
                                           + feedstockproduction_ef_n2O_ethanol_baseline*gwp_n2O

                                           + distribution_ef_co2_ethanol_baseline
                                           + distribution_ef_ch4_ethanol_baseline*gwp_ch4
                                           + distribution_ef_n2O_ethanol_baseline*gwp_n2O

                                           + processing_ef_co2_ethanol_baseline
                                           + processing_ef_ch4_ethanol_baseline*gwp_ch4
                                           + processing_ef_n2O_ethanol_baseline*gwp_n2O)

                                           + stoveproduction_ef_co2_ethanol_baseline
                                           + stoveproduction_ef_ch4_ethanol_baseline*gwp_ch4
                                           + stoveproduction_ef_n2O_ethanol_baseline*gwp_n2O)


    # Baseline  Commercial Pellets

    be_pellets_commercial_gillwiehl=prc_pellets_baseline_commercial*(ncv_pellets_baseline*kg_pellets_baseline_commercial/1000*((n_fNRB*co2_ef_pellets)
                                           + nonco2_ef_pellets

                                           + feedstockproduction_ef_co2_pellets_baseline
                                           + feedstockproduction_ef_ch4_pellets_baseline*gwp_ch4
                                           + feedstockproduction_ef_n2O_pellets_baseline*gwp_n2O

                                           + distribution_ef_co2_pellets_baseline
                                           + distribution_ef_ch4_pellets_baseline*gwp_ch4
                                           + distribution_ef_n2O_pellets_baseline*gwp_n2O

                                           + processing_ef_co2_pellets_baseline
                                           + processing_ef_ch4_pellets_baseline*gwp_ch4
                                           + processing_ef_n2O_pellets_baseline*gwp_n2O)

                                           + stoveproduction_ef_co2_pellets_baseline
                                           + stoveproduction_ef_ch4_pellets_baseline*gwp_ch4
                                           + stoveproduction_ef_n2O_pellets_baseline*gwp_n2O)
    be_total_commercial_gillwiehl=be_firewood_commercial_gillwiehl+ be_charcoal_commercial_gillwiehl + be_LPG_commercial_gillwiehl + be_electric_gillwiehl + be_ethanol_commercial_gillwiehl + be_pellets_commercial_gillwiehl
    be_total_commercial_gillwiehl

    # Project Domestic Emissions
    # Sum all DOMESTIC project emissions from each baseline fuel/technology
    # Project Firewood

    pe_firewood_gillwiehl=prc_firewood_project*(ncv_firewood_project*kg_firewood_project/1000*((n_fNRB*co2_ef_firewood_project)
                                           + nonco2_ef_firewood_project

                                           + feedstockproduction_ef_co2_firewood_project
                                           + feedstockproduction_ef_ch4_firewood_project*gwp_ch4_project
                                           + feedstockproduction_ef_n2O_firewood_project*gwp_n2O_project

                                           + distribution_ef_co2_firewood_project
                                           + distribution_ef_ch4_firewood_project*gwp_ch4_project
                                           + distribution_ef_n2O_firewood_project*gwp_n2O_project

                                           + processing_ef_co2_firewood_project
                                           + processing_ef_ch4_firewood_project*gwp_ch4_project
                                           + processing_ef_n2O_firewood_project*gwp_n2O_project)

                                           + stoveproduction_ef_co2_firewood_project
                                           + stoveproduction_ef_ch4_firewood_project*gwp_ch4_project
                                           + stoveproduction_ef_n2O_firewood_project*gwp_n2O_project)


    # Project Charcoal

    pe_charcoal_gillwiehl=prc_charcoal_project*(ncv_charcoal_project*kg_charcoal_project/1000*((n_fNRB*co2_ef_charcoal_project)
                                           + nonco2_ef_charcoal_project

                                           + feedstockproduction_ef_co2_charcoal_project
                                           + feedstockproduction_ef_ch4_charcoal_project*gwp_ch4_project
                                           + feedstockproduction_ef_n2O_charcoal_project*gwp_n2O_project

                                           + distribution_ef_co2_charcoal_project
                                           + distribution_ef_ch4_charcoal_project*gwp_ch4_project
                                           + distribution_ef_n2O_charcoal_project*gwp_n2O_project

                                           + processing_ef_co2_charcoal_project
                                           + processing_ef_ch4_charcoal_project*gwp_ch4_project
                                           + processing_ef_n2O_charcoal_project*gwp_n2O_project)

                                           + stoveproduction_ef_co2_charcoal_project
                                           + stoveproduction_ef_ch4_charcoal_project*gwp_ch4_project
                                           + stoveproduction_ef_n2O_charcoal_project*gwp_n2O_project)


    # Project LPG

    pe_LPG_gillwiehl=prc_lpg_project*(ncv_LPG_project*kg_lpg_project/1000*(co2_ef_LPG_project+nonco2_ef_LPG_project

                                           + feedstockproduction_ef_co2_LPG_project
                                           + feedstockproduction_ef_ch4_LPG_project*gwp_ch4_project
                                           + feedstockproduction_ef_n2O_LPG_project*gwp_n2O_project

                                           + distribution_ef_co2_LPG_project
                                           + distribution_ef_ch4_LPG_project*gwp_ch4_project
                                           + distribution_ef_n2O_LPG_project*gwp_n2O_project

                                           + processing_ef_co2_LPG_project
                                           + processing_ef_ch4_LPG_project*gwp_ch4_project
                                           + processing_ef_n2O_LPG_project*gwp_n2O_project)

                                           + stoveproduction_ef_co2_LPG_project
                                           + stoveproduction_ef_ch4_LPG_project*gwp_ch4_project
                                           + stoveproduction_ef_n2O_LPG_project*gwp_n2O_project)


    # Project kerosene

    pe_kerosene_gillwiehl=prc_kerosene_project*(ncv_kerosene_project*kg_kerosene_project/1000*(co2_ef_kerosene_project+nonco2_ef_kerosene_project

                                           + feedstockproduction_ef_co2_kerosene_project
                                           + feedstockproduction_ef_ch4_kerosene_project*gwp_ch4_project
                                           + feedstockproduction_ef_n2O_kerosene_project*gwp_n2O_project

                                           + distribution_ef_co2_kerosene_project
                                           + distribution_ef_ch4_kerosene_project*gwp_ch4_project
                                           + distribution_ef_n2O_kerosene_project*gwp_n2O_project

                                           + processing_ef_co2_kerosene_project
                                           + processing_ef_ch4_kerosene_project*gwp_ch4_project
                                           + processing_ef_n2O_kerosene_project*gwp_n2O_project)

                                           + stoveproduction_ef_co2_kerosene_project
                                           + stoveproduction_ef_ch4_kerosene_project*gwp_ch4_project
                                           + stoveproduction_ef_n2O_kerosene_project*gwp_n2O_project)


    # Project electricity

    pe_electric_gillwiehl=prc_electric_project*(kWh_electric_project/1000*(co2_ef_electric_project+nonco2_ef_electric_project

                                           + feedstockproduction_ef_co2_electric_project
                                           + feedstockproduction_ef_ch4_electric_project*gwp_ch4_project
                                           + feedstockproduction_ef_n2O_electric_project*gwp_n2O_project

                                           + distribution_ef_co2_electric_project
                                           + distribution_ef_ch4_electric_project*gwp_ch4_project
                                           + distribution_ef_n2O_electric_project*gwp_n2O_project

                                           + processing_ef_co2_electric_project
                                           + processing_ef_ch4_electric_project*gwp_ch4_project
                                           + processing_ef_n2O_electric_project*gwp_n2O_project)

                                           + stoveproduction_ef_co2_electric_project
                                           + stoveproduction_ef_ch4_electric_project*gwp_ch4_project
                                           + stoveproduction_ef_n2O_electric_project*gwp_n2O_project)


    # Project ethanol

    pe_ethanol_gillwiehl=prc_ethanol_project*(ncv_ethanol_project*l_ethanol_project*kg_per_liter_ethanol_project/1000*(co2_ef_ethanol_project+nonco2_ef_ethanol_project

                                           + feedstockproduction_ef_co2_ethanol_project
                                           + feedstockproduction_ef_ch4_ethanol_project*gwp_ch4_project
                                           + feedstockproduction_ef_n2O_ethanol_project*gwp_n2O_project

                                           + distribution_ef_co2_ethanol_project
                                           + distribution_ef_ch4_ethanol_project*gwp_ch4_project
                                           + distribution_ef_n2O_ethanol_project*gwp_n2O_project

                                           + processing_ef_co2_ethanol_project
                                           + processing_ef_ch4_ethanol_project*gwp_ch4_project
                                           + processing_ef_n2O_ethanol_project*gwp_n2O_project)

                                           + stoveproduction_ef_co2_ethanol_project
                                           + stoveproduction_ef_ch4_ethanol_project*gwp_ch4_project
                                           + stoveproduction_ef_n2O_ethanol_project*gwp_n2O_project)


    # Project pellets

    pe_pellets_gillwiehl=prc_pellets_project*(ncv_pellets_project*kg_pellets_project/1000*((n_fNRB*co2_ef_pellets_project)
                                           + nonco2_ef_pellets_project

                                           + feedstockproduction_ef_co2_pellets_project
                                           + feedstockproduction_ef_ch4_pellets_project*gwp_ch4_project
                                           + feedstockproduction_ef_n2O_pellets_project*gwp_n2O_project

                                           + distribution_ef_co2_pellets_project
                                           + distribution_ef_ch4_pellets_project*gwp_ch4_project
                                           + distribution_ef_n2O_pellets_project*gwp_n2O_project

                                           + processing_ef_co2_pellets_project
                                           + processing_ef_ch4_pellets_project*gwp_ch4_project
                                           + processing_ef_n2O_pellets_project*gwp_n2O_project)

                                           + stoveproduction_ef_co2_pellets_project
                                           + stoveproduction_ef_ch4_pellets_project*gwp_ch4_project
                                           + stoveproduction_ef_n2O_pellets_project*gwp_n2O_project)
    pe_total_domestic_gillwiehl=pe_firewood_gillwiehl+ pe_charcoal_gillwiehl + pe_LPG_gillwiehl + pe_electric_gillwiehl + pe_ethanol_gillwiehl + pe_pellets_gillwiehl
    pe_total_domestic_gillwiehl

    # Project Commercial Emissions
    # Sum all project commercial emissions from each baseline fuel/technology

    # Project Commercial Firewood
# Project Commercial Firewood

    pe_firewood_commercial_gillwiehl=prc_firewood_project_commercial*(ncv_firewood_project*kg_firewood_project_commercial/1000*((n_fNRB*co2_ef_firewood)
                                        + nonco2_ef_firewood

                                        + feedstockproduction_ef_co2_firewood_project
                                        + feedstockproduction_ef_ch4_firewood_project*gwp_ch4
                                        + feedstockproduction_ef_n2O_firewood_project*gwp_n2O

                                        + distribution_ef_co2_firewood_project
                                        + distribution_ef_ch4_firewood_project*gwp_ch4
                                        + distribution_ef_n2O_firewood_project*gwp_n2O

                                        + processing_ef_co2_firewood_project
                                        + processing_ef_ch4_firewood_project*gwp_ch4
                                        + processing_ef_n2O_firewood_project*gwp_n2O)

                                        + stoveproduction_ef_co2_firewood_project
                                        + stoveproduction_ef_ch4_firewood_project*gwp_ch4
                                        + stoveproduction_ef_n2O_firewood_project*gwp_n2O)


    # Project Commercial Charcoal

    pe_charcoal_commercial_gillwiehl=prc_charcoal_project_commercial*(ncv_charcoal_project*kg_charcoal_project_commercial/1000*((n_fNRB*co2_ef_charcoal)
                                           + nonco2_ef_charcoal

                                           + feedstockproduction_ef_co2_charcoal_project
                                           + feedstockproduction_ef_ch4_charcoal_project*gwp_ch4
                                           + feedstockproduction_ef_n2O_charcoal_project*gwp_n2O

                                           + distribution_ef_co2_charcoal_project
                                           + distribution_ef_ch4_charcoal_baseline*gwp_ch4
                                           + distribution_ef_n2O_charcoal_baseline*gwp_n2O

                                           + processing_ef_co2_charcoal_project
                                           + processing_ef_ch4_charcoal_project*gwp_ch4
                                           + processing_ef_n2O_charcoal_project*gwp_n2O)

                                           + stoveproduction_ef_co2_charcoal_project
                                           + stoveproduction_ef_ch4_charcoal_project*gwp_ch4
                                           + stoveproduction_ef_n2O_charcoal_project*gwp_n2O)


    #  Project Commercial LPG

    pe_LPG_commercial_gillwiehl=prc_lpg_project_commercial*(ncv_LPG_project*kg_lpg_project_commercial/1000*(co2_ef_LPG +nonco2_ef_LPG

                                           + feedstockproduction_ef_co2_charcoal_project
                                           + feedstockproduction_ef_ch4_charcoal_project*gwp_ch4
                                           + feedstockproduction_ef_n2O_charcoal_project*gwp_n2O

                                           + distribution_ef_co2_charcoal_project
                                           + distribution_ef_ch4_charcoal_project*gwp_ch4
                                           + distribution_ef_n2O_charcoal_project*gwp_n2O

                                           + processing_ef_co2_charcoal_project
                                           + processing_ef_ch4_charcoal_project*gwp_ch4
                                           + processing_ef_n2O_charcoal_project*gwp_n2O)

                                           + stoveproduction_ef_co2_charcoal_project
                                           + stoveproduction_ef_ch4_charcoal_project*gwp_ch4
                                           + stoveproduction_ef_n2O_charcoal_project*gwp_n2O)


    # Project Commercial Kerosene

    pe_kerosene_commercial_gillwiehl=prc_kerosene_project_commercial*(ncv_kerosene_project* kg_kerosene_project_commercial/1000*(co2_ef_kerosene +nonco2_ef_kerosene

                                           + feedstockproduction_ef_co2_kerosene_project
                                           + feedstockproduction_ef_ch4_kerosene_project*gwp_ch4
                                           + feedstockproduction_ef_n2O_kerosene_project*gwp_n2O

                                           + distribution_ef_co2_kerosene_project
                                           + distribution_ef_ch4_kerosene_project*gwp_ch4
                                           + distribution_ef_n2O_kerosene_project*gwp_n2O

                                           + processing_ef_co2_kerosene_project
                                           + processing_ef_ch4_kerosene_project*gwp_ch4
                                           + processing_ef_n2O_kerosene_project*gwp_n2O)

                                           + stoveproduction_ef_co2_kerosene_project
                                           + stoveproduction_ef_ch4_kerosene_project*gwp_ch4
                                           + stoveproduction_ef_n2O_kerosene_project*gwp_n2O)


    # ProjectCommercial Electricity

    pe_electric_commercial_gillwiehl=prc_electric_project_commercial*(kWh_electric_project_commercial/1000*(co2_ef_electric+nonco2_ef_electric

                                           + feedstockproduction_ef_co2_electric_project
                                           + feedstockproduction_ef_ch4_electric_project*gwp_ch4
                                           + feedstockproduction_ef_n2O_electric_project*gwp_n2O

                                           + distribution_ef_co2_electric_project
                                           + distribution_ef_ch4_electric_project*gwp_ch4
                                           + distribution_ef_n2O_electric_project*gwp_n2O

                                           + processing_ef_co2_electric_project
                                           + processing_ef_ch4_electric_project*gwp_ch4
                                           + processing_ef_n2O_electric_project*gwp_n2O)

                                           + stoveproduction_ef_co2_electric_project
                                           + stoveproduction_ef_ch4_electric_project*gwp_ch4
                                           + stoveproduction_ef_n2O_electric_project*gwp_n2O)


    # Project Commercial Ethanol

    pe_ethanol_commercial_gillwiehl=prc_ethanol_project_commercial*(ncv_ethanol_project*l_ethanol_project_commercial*kg_per_liter_ethanol_project/1000*(co2_ef_ethanol+nonco2_ef_ethanol

                                           + feedstockproduction_ef_co2_ethanol_project
                                           + feedstockproduction_ef_ch4_ethanol_project*gwp_ch4
                                           + feedstockproduction_ef_n2O_ethanol_project*gwp_n2O

                                           + distribution_ef_co2_ethanol_project
                                           + distribution_ef_ch4_ethanol_project*gwp_ch4
                                           + distribution_ef_n2O_ethanol_project*gwp_n2O

                                           + processing_ef_co2_ethanol_project
                                           + processing_ef_ch4_ethanol_project*gwp_ch4
                                           + processing_ef_n2O_ethanol_project*gwp_n2O)

                                           + stoveproduction_ef_co2_ethanol_project
                                           + stoveproduction_ef_ch4_ethanol_project*gwp_ch4
                                           + stoveproduction_ef_n2O_ethanol_project*gwp_n2O)


    # Project Commercial Pellets

    pe_pellets_commercial_gillwiehl=prc_pellets_project_commercial*(ncv_pellets_project*kg_pellets_project_commercial/1000*((n_fNRB*co2_ef_pellets)
                                           + nonco2_ef_pellets

                                           + feedstockproduction_ef_co2_pellets_project
                                           + feedstockproduction_ef_ch4_pellets_project*gwp_ch4
                                           + feedstockproduction_ef_n2O_pellets_project*gwp_n2O

                                           + distribution_ef_co2_pellets_project
                                           + distribution_ef_ch4_pellets_project*gwp_ch4
                                           + distribution_ef_n2O_pellets_project*gwp_n2O

                                           + processing_ef_co2_pellets_project
                                           + processing_ef_ch4_pellets_project*gwp_ch4
                                           + processing_ef_n2O_pellets_project*gwp_n2O)

                                           + stoveproduction_ef_co2_pellets_project
                                           + stoveproduction_ef_ch4_pellets_project*gwp_ch4
                                           + stoveproduction_ef_n2O_pellets_project*gwp_n2O)

    pe_total_commercial_gillwiehl=pe_firewood_commercial_gillwiehl+ pe_charcoal_commercial_gillwiehl + pe_LPG_commercial_gillwiehl + pe_electric_gillwiehl + pe_ethanol_commercial_gillwiehl + pe_pellets_commercial_gillwiehl
    pe_total_commercial_gillwiehl

    be_total_agw=be_total_domestic_gillwiehl

    be_total_agw=be_total_agw+be_total_commercial_gillwiehl

    pe_total_agw=pe_total_domestic_gillwiehl
    pe_total_agw=pe_total_agw+pe_total_commercial_gillwiehl



    if df.protocol_type[i]=='GS Metered'and df.stove_year_cohort[i]!='commercial':

        be_total_agw= be_total_agw*project_adoption*project_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*project_adoption*project_usage*(1-project_stacking)

    if df.protocol_type[i]=='GS TPDDTEC'and df.protocol_id[i]!='GS2758'and df.stove_year_cohort[i]!='commercial':

        be_total_agw= be_total_agw*project_adoption*kpt_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*project_adoption*kpt_usage*(1-project_stacking)

    if df.protocol_type[i]=='GS TPDDTEC'and df.stove_year_cohort[i]=='commercial':

        be_total_agw= be_total_agw*project_adoption*project_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*project_adoption*project_usage*(1-project_stacking)


    if df.protocol_type[i]=='GS Simplified'and df.stove_year_cohort[i]!='commercial':

        be_total_agw= be_total_agw*project_adoption*n_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*project_adoption*n_usage*(1-project_stacking)

    if df.protocol_type[i]=='CDM AMS II G'and df.protocol_id[i]!='GS10974'and df.stove_year_cohort[i]!='commercial':

        be_total_agw= be_total_agw*project_adoption*n_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*project_adoption*n_usage*(1-project_stacking)

    if df.protocol_type[i]=='CDM AMS I E'and df.stove_year_cohort[i]!='commercial':

        be_total_agw= be_total_agw*project_adoption*n_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*project_adoption*n_usage*(1-project_stacking)

    if df.protocol_id[i]=='GS2758'and df.stove_year_cohort[i]!='commercial':

        be_total_agw= be_total_agw*project_adoption*kpt_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*project_adoption*kpt_usage*(1-project_stacking)


    if df.protocol_id[i]=='GS10974'and df.stove_year_cohort[i]!='commercial':

        be_total_agw= be_total_agw*project_adoption*req_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*project_adoption*req_usage*(1-project_stacking)

    if df.protocol_type[i]=='CDM AMS I E'and df.stove_year_cohort[i]=='commercial':

        be_total_agw= be_total_agw*project_adoption*project_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*project_adoption*project_usage*(1-project_stacking)


    if df.protocol_type[i]=='CDM AMS II G'and df.stove_year_cohort[i]=='commercial':

        be_total_agw= be_total_agw*project_adoption_commercial*project_usage_commercial*(1-project_stacking_commercial)

        pe_total_agw= pe_total_agw*project_adoption_commercial*project_usage_commercial*(1-project_stacking_commercial)


    # Leakage if not default
    le_total_agw=(leakage_tCO2e+other_tCO2e_reduction)/(n_devices_domestic+n_devices_commercial)/(project_days+project_days_commercial) # Project Emission Reductions (PERs) in (tCO2e)





    # Totally for all
    PEs_agw=(pe_total_agw)# Project Emission Reductions (PERs) in (tCO2e)
    BEs_agw=(be_total_agw)# Project Emission Reductions (PERs) in (tCO2e)

    VERs_agw=(be_total_agw-pe_total_agw-le_total_agw)*more_than_one_stove_adj*leakage_default


    # Protocol Analysis:



    all_VERS_agw.append(VERs_agw)
    ALL_N=ALL_N+1



    # Protocol Analysis:

    #GS
    if df.protocol_type[i]=='GS Metered':
        GS_Meter_N=GS_Meter_N+1
        GS_Meter_VERs_agw.append(VERs_agw)

    #GS
    if df.protocol_type[i]=='GS TPDDTEC':
        GS_N=GS_N+1
        GS_VERs_agw.append(VERs_agw)


    # CDM II G

    if df.protocol_type[i]=='CDM AMS II G':
        CDM_II_G_N=CDM_II_G_N+1
        CDM_II_G_VERs_agw.append(VERs_agw)

    # CDM I E
    if df.protocol_type[i]=='CDM AMS I E':
        CDM_I_E_N=CDM_I_E_N+1
        CDM_I_E_VERs_agw.append(VERs_agw)

    #bangladesh
    if df.country[i]=='Bangladesh':
        bangladesh_N=bangladesh_N+1
        bangladesh_VERs_agw.append(VERs_agw)

    #burkina faso
    if df.country[i]=='Burkina Faso':
        burkinafaso_N=burkinafaso_N+1
        burkinafaso_VERs_agw.append(VERs_agw)

    #cameroon
    if df.country[i]=='Cameroon':
        cameroon_N=cameroon_N+1
        cameroon_VERs_agw.append(VERs_agw)


    #china
    if df.country[i]=='China':
        china_N=china_N+1
        china_VERs_agw.append(VERs_agw)

    #ethiopia
    if df.country[i]=='Ethiopia':
        ethiopia_N=ethiopia_N+1
        ethiopia_VERs_agw.append(VERs_agw)

    #ghana
    if df.country[i]=='Ghana':
        ghana_N=ghana_N+1
        ghana_VERs_agw.append(VERs_agw)

    #burundi                                                                        ## SY added
    if df.country[i]=='Burundi':
        burundi_N=burundi_N+1
        burundi_VERs_agw.append(VERs_agw)


    #guatemala
    if df.country[i]=='Guatemala':
        guatemala_N=guatemala_N+1
        guatemala_VERs_agw.append(VERs_agw)

    #haiti
    if df.country[i]=='Haiti':
        haiti_N=haiti_N+1
        haiti_VERs_agw.append(VERs_agw)

    #honduras
    if df.country[i]=='Honduras':
        honduras_N=honduras_N+1
        honduras_VERs_agw.append(VERs_agw)

    #india
    if df.country[i]=='India':
        india_N=india_N+1
        india_VERs_agw.append(VERs_agw)

    #kenya
    if df.country[i]=='Kenya':
        kenya_N=kenya_N+1
        kenya_VERs_agw.append(VERs_agw)

    #lesotho
    if df.country[i]=='Lesotho':
        lesotho_N=lesotho_N+1
        lesotho_VERs_agw.append(VERs_agw)

    #malawi
    if df.country[i]=='Malawi':
        malawi_N=malawi_N+1
        malawi_VERs_agw.append(VERs_agw)

    #mali
    if df.country[i]=='Mali':
        mali_N=mali_N+1
        mali_VERs_agw.append(VERs_agw)

    #mexico
    if df.country[i]=='Mexico':
        mexico_N=mexico_N+1
        mexico_VERs_agw.append(VERs_agw)

    #mozambique
    if df.country[i]=='Mozambique':
        mozambique_N=mozambique_N+1
        mozambique_VERs_agw.append(VERs_agw)

    #myanmar
    if df.country[i]=='Myanmar':
        myanmar_N=myanmar_N+1
        myanmar_VERs_agw.append(VERs_agw)

    #nepal
    if df.country[i]=='Nepal':
        nepal_N=nepal_N+1
        nepal_VERs_agw.append(VERs_agw)

    #nigeria
    if df.country[i]=='Nigeria':
        nigeria_N=nigeria_N+1
        nigeria_VERs_agw.append(VERs_agw)

    #peru
    if df.country[i]=='Peru':
        peru_N=peru_N+1
        peru_VERs_agw.append(VERs_agw)


    #rwanda
    if df.country[i]=='Rwanda':
        rwanda_N=rwanda_N+1
        rwanda_VERs_agw.append(VERs_agw)

    #sudan
    if df.country[i]=='Sudan':
        sudan_N=sudan_N+1
        sudan_VERs_agw.append(VERs_agw)

    #uganda
    if df.country[i]=='Uganda':
        uganda_N=uganda_N+1
        uganda_VERs_agw.append(VERs_agw)

    #zambia
    if df.country[i]=='Zambia':
        zambia_N=zambia_N+1
        zambia_VERs_agw.append(VERs_agw)

    #firewood
    if df.type[i]=='firewood':
        firewood_N=firewood_N+1
        firewood_VERs_agw.append(VERs_agw)

    #charcoal
    if df.type[i]=='charcoal':
        charcoal_N=charcoal_N+1
        charcoal_VERs_agw.append(VERs_agw)

    #biomass
    if df.type[i]=='biomass':
        biomass_N=charcoal_N+1
        biomass_VERs_agw.append(VERs_agw)

    #switch
    if df.type[i]=='switch':
        switch_N=switch_N+1
        switch_VERs_agw.append(VERs_agw)

     #gs_lpg
    if df.stove[i]=='gs_lpg':
        gs_lpg_N=gs_lpg_N+1
        gs_lpg_VERs_agw.append(VERs_agw)

    #gs_imp_firewood
    if df.stove[i]=='gs_imp_firewood':
        gs_imp_firewood_N=gs_imp_firewood_N+1
        gs_imp_firewood_VERs_agw.append(VERs_agw)

    #gs_imp_charcoal
    if df.stove[i]=='gs_imp_charcoal':
        gs_imp_charcoal_N=gs_imp_charcoal_N+1
        gs_imp_charcoal_VERs_agw.append(VERs_agw)

    #cdm_imp_firewood
    if df.stove[i]=='cdm_imp_firewood':
        cdm_imp_firewood_N=cdm_imp_firewood_N+1
        cdm_imp_firewood_VERs_agw.append(VERs_agw)

    #cdm_imp_charcoal
    if df.stove[i]=='cdm_imp_charcoal':
        cdm_imp_charcoal_N=cdm_imp_charcoal_N+1
        cdm_imp_charcoal_VERs_agw.append(VERs_agw)

    #gs_simple_firewood
    if df.stove[i]=='gs_simple_firewood':
        gs_simple_firewood_N=gs_simple_firewood_N+1
        gs_simple_firewood_VERs_agw.append(VERs_agw)

    print(protocol_id)

    #Times Over Credited
    ratio_analysis_project=VERs_agw/VERs
    ratio_analysis_project_mean=np.mean(VERs_agw)/VERs*100
    ratio_analysis_project_mean
    ratio_analysis_project_mean_compilation.append(ratio_analysis_project_mean)


    ratio_project_analysis=VERs/VERs_agw
    ratio_project_analysis_mean=VERs/np.mean(VERs_agw)
    ratio_project_analysis_mean
    ratio_project_analysis_mean_compilation.append(ratio_project_analysis_mean)
    z=1.96

    sd_VERs_agw=np.std(VERs_agw)
    sd_ratio_analysis_project=np.std(ratio_analysis_project)
    sd_ratio_project_analysis=np.std(ratio_project_analysis)


    ratio_analysis_project_lower=ratio_analysis_project_mean - z*sd_ratio_analysis_project/np.sqrt(N)
    ratio_analysis_project_upper=ratio_analysis_project_mean + z*sd_ratio_analysis_project/np.sqrt(N)

    ratio_analysis_project_lower_compilation.append(ratio_analysis_project_lower)
    ratio_analysis_project_upper_compilation.append(ratio_analysis_project_upper)


    ratio_project_analysis_lower=ratio_project_analysis_mean - z*sd_ratio_project_analysis/np.sqrt(N)
    ratio_project_analysis_upper=ratio_project_analysis_mean + z*sd_ratio_project_analysis/np.sqrt(N)

    ratio_project_analysis_lower_compilation.append(ratio_project_analysis_lower)
    ratio_project_analysis_upper_compilation.append(ratio_project_analysis_upper)

    print('We find that this project reports emissions which are',round(ratio_project_analysis_mean,2),'[',round(ratio_project_analysis_lower,2),' , ',round(ratio_project_analysis_upper,2),']','times overcredited.')
    print('Our analysis finds emissions which are',round(ratio_analysis_project_mean,2),'[',round(ratio_analysis_project_lower,2),' , ',round(ratio_analysis_project_upper,2),']','% of the verified credits in the PDD.')
    print('------------------------------------------------------------')
    i=i+1

PoA 10471 CPA1
We find that this project reports emissions which are 1.96 [ 1.95  ,  1.98 ] times overcredited.
Our analysis finds emissions which are 50.9 [ 50.89  ,  50.9 ] % of the verified credits in the PDD.
------------------------------------------------------------
PoA 10471 CPA2
We find that this project reports emissions which are 1.96 [ 1.95  ,  1.98 ] times overcredited.
Our analysis finds emissions which are 50.9 [ 50.89  ,  50.9 ] % of the verified credits in the PDD.
------------------------------------------------------------
PoA 10471 CPA3
We find that this project reports emissions which are 1.96 [ 1.95  ,  1.98 ] times overcredited.
Our analysis finds emissions which are 50.9 [ 50.89  ,  50.9 ] % of the verified credits in the PDD.
------------------------------------------------------------
PoA 10471 CPA4
We find that this project reports emissions which are 1.96 [ 1.95  ,  1.98 ] times overcredited.
Our analysis finds emissions which are 50.9 [ 50.89  ,  50.9 ] % o

In [16]:
ratio_project_analysis_mean_compilation=pd.DataFrame(ratio_project_analysis_mean_compilation)
ratio_project_analysis_lower_compilation=pd.DataFrame(ratio_project_analysis_lower_compilation)
ratio_project_analysis_upper_compilation=pd.DataFrame(ratio_project_analysis_upper_compilation)
ratio_analysis_project_mean_compilation=pd.DataFrame(ratio_analysis_project_mean_compilation)
ratio_analysis_project_lower_compilation=pd.DataFrame(ratio_analysis_project_lower_compilation)
ratio_analysis_project_upper_compilation=pd.DataFrame(ratio_analysis_project_upper_compilation)

# Total over-crediting

In [17]:
df.columns

Index(['protocol_type', 'protocol_version', 'PoA', 'protocol_id',
       'monitoring_period', 'stove_year_cohort', 'total_verified_credits',
       'type', 'verified_credits', 'verified_credits_per_stove',
       ...
       'project_adoption_commercial', 'project_usage_commercial',
       'leakage_binary', 'leakage_default', 'leakage_tCO2e',
       'more_than_one_stove_adj', 'rebound', 'other_tCO2e_reduction',
       'project_stacking', 'project_stacking_commercial'],
      dtype='object', length=332)

In [18]:
PoAs=pd.DataFrame(df['PoA'])
ids=pd.DataFrame(df['protocol_id'])
category=pd.DataFrame(df['stove'])
country=pd.DataFrame(df['country'])
total_credits=pd.DataFrame(df['total_verified_credits'])
stove_days=pd.DataFrame(df['stove_days'])


df_info=PoAs.join(ids)
df_info=df_info.join(category)
df_info=df_info.join(country)
df_info=df_info.join(total_credits)
df_info=df_info.join(stove_days)
df_info=pd.DataFrame(df_info)
df_info=pd.concat([df_info[1:].reset_index(drop=True),ratio_project_analysis_lower_compilation], axis=1)
df_info= pd.concat([df_info.reset_index(drop=True),ratio_project_analysis_mean_compilation], axis=1)
df_info= pd.concat([df_info.reset_index(drop=True),ratio_project_analysis_upper_compilation], axis=1)
df_info.columns = ['PoA','protocol_id', 'stove', 'country','total_verified_credits','stove_days','lower','mean','upper']
df_info['stove_days_mean_overcrediting']=df_info['mean']*df_info['stove_days']
df_info
df_project_weighted_stove_days_numerator=df_info.groupby('protocol_id')['stove_days_mean_overcrediting'].sum()
df_project_weighted_stove_days_numerator=pd.DataFrame(df_project_weighted_stove_days_numerator)


df_project_weighted_stove_days_denominator=df_info.groupby('protocol_id')['stove_days'].sum()
df_project_weighted_stove_days_denominator=pd.DataFrame(df_project_weighted_stove_days_denominator)
df_project_weighted_stove_days=df_project_weighted_stove_days_numerator['stove_days_mean_overcrediting']/df_project_weighted_stove_days_denominator['stove_days']
df_project_weighted_stove_days=pd.DataFrame(df_project_weighted_stove_days)
df_project_weighted_stove_days.mean() #This is the average project over-crediting
print(df_project_weighted_stove_days.mean())
df_project_weighted_stove_days
protocol_ids=pd.DataFrame(df_info['protocol_id'].unique())
protocol_ids.columns=['protocol_id']

df_total=df_project_weighted_stove_days.merge(df_info,left_on='protocol_id', right_on='protocol_id',how='left')
df_total.columns=['protocol_id','project_over_crediting','PoA','category','country','total_verified_credits','stove_days','lower','mean','upper','stove_days_mean_overcrediting']

df_total['over_credited_total']=df_total['total_verified_credits']/df_total['project_over_crediting']
total_verified_credits=df_total['total_verified_credits'].unique().sum()
total_over_crediting =df_total['over_credited_total'].unique().sum()
reported_value=total_verified_credits/total_over_crediting
print(total_verified_credits/total_over_crediting)
sd=df_total['total_verified_credits'].unique().std()/df_total['over_credited_total'].unique().std()
ci=sd*1.96/np.sqrt(51)
ci
ci_lower=reported_value-ci
ci_upper=reported_value+ci
print(reported_value,'[',ci_lower,',',ci_upper,']')

0    1.775603
dtype: object
1.72241346805745
1.72241346805745 [ 1.174283212769351 , 2.270543723345549 ]


# PoA, country and Methodology-stove combination analysis

In [20]:
import pandas as pd
import numpy as np
from scipy import stats

def weighted_std(values, weights):
    """Calculate weighted standard deviation."""
    average = np.average(values, weights=weights)
    variance = np.average((values - average) ** 2, weights=weights)
    return np.sqrt(variance)

def weighted_std(values, weights):
    """Calculate weighted standard deviation."""
    average = np.average(values, weights=weights)
    variance = np.average((values - average) ** 2, weights=weights)
    return np.sqrt(variance)

def calculate_weighted_statistics(df, group_by_col):
    """
    Calculate weighted statistics grouped by specified column.

    Parameters:
    df (pandas.DataFrame): DataFrame with columns for grouping,
                         'total_verified_credits', and 'mean'
    group_by_col (str): Name of the column to group by

    Returns:
    pandas.DataFrame: Weighted statistics by group including:
                     - weighted average
                     - weighted standard deviation
                     - 95% confidence intervals
    """
    # Initialize empty lists to store results
    groups = []
    w_means = []
    w_stds = []
    lower_cis = []
    upper_cis = []

    # Calculate statistics for each group
    for group in df[group_by_col].unique():
        group_data = df[df[group_by_col] == group]
        weights = group_data['total_verified_credits']
        values = group_data['mean']

        # Calculate weighted mean
        w_mean = np.average(values, weights=weights)

        # Calculate weighted standard deviation
        w_std = weighted_std(values, weights)

        # Calculate standard error
        n = len(group_data)
        se = w_std / np.sqrt(n)

        # Calculate 95% confidence intervals
        ci = stats.t.interval(confidence=0.95, df=n-1, loc=w_mean, scale=se)

        # Store results
        groups.append(group)
        w_means.append(w_mean)
        w_stds.append(w_std)
        lower_cis.append(ci[0])
        upper_cis.append(ci[1])

    # Create results DataFrame
    results = pd.DataFrame({
        group_by_col: groups,
        'total': w_means,
        'sd': w_stds,
        'lower': lower_cis,
        'upper': upper_cis
    })

    # Round numeric columns to 4 decimal places
    numeric_columns = ['total', 'sd', 'lower', 'upper']
    results[numeric_columns] = results[numeric_columns].round(4)

    # Sort by group column
    results = results.sort_values(group_by_col).reset_index(drop=True)

    return results



In [21]:
df_PoA_final =calculate_weighted_statistics(df_total, 'PoA')
df_PoA_final

,PoA,total,sd,lower,upper
0,5341,1.7413,0.2318,1.6260,1.8565
1,5342,1.6539,0.2090,1.5832,1.7246
2,6864,1.3792,0.5856,0.7646,1.9938
3,7359,1.9648,0.0000,1.9648,1.9648
4,7997,0.9929,0.2430,0.8680,1.1179
5,8480,1.6658,0.1726,1.3911,1.9405
6,9007,1.5775,0.3751,1.4320,1.7229
7,9265,1.7642,0.0896,1.7234,1.8050
8,9956,1.6223,0.3686,1.5317,1.7129
9,10008,1.8131,0.0000,1.8131,1.8131


In [22]:
df_country_final =calculate_weighted_statistics(df_total, 'country')
df_country_final

,country,total,sd,lower,upper
0,Bangladesh,1.9648,0.0000,1.9648,1.9648
1,Burundi,1.6823,0.1942,1.1998,2.1648
2,Cambodia,1.8416,0.0102,1.8322,1.8510
3,Fiji,1.9648,0.0000,1.9648,1.9648
4,Ghana,1.8694,0.1822,1.7757,1.9630
5,Guatemala,1.6658,0.1726,1.3911,1.9405
6,Kenya,1.7588,0.2509,1.6945,1.8230
7,Lao,1.8614,0.0417,1.8177,1.9052
8,Madagascar,1.9648,0.0000,1.9648,1.9648
9,Malawi,1.6183,0.3788,1.4358,1.8009


In [23]:
df_combination_final=calculate_weighted_statistics(df_total, 'category')
df_combination_final

,category,total,sd,lower,upper
0,cdm_imp_charcoal,1.7562,0.2113,1.6857,1.8266
1,cdm_imp_ethanol,1.4340,0.0904,1.3218,1.5463
2,cdm_imp_firewood,1.8545,0.2576,1.8288,1.8802
3,ethanol,1.9567,0.0982,1.9000,2.0133


In [24]:
df_combination_final.to_excel('all_results_usage_categories.xlsx')   ## SY updated

In [26]:
category1 = ['All', 'GS-Metered-Pellets', 'GS-LPG', 'GS-Firewood','GS-Simplified Firewood','GS-Charcoal', 'CDM-Charcoal', 'CDM_Ethanol', 'CDM-Firewood', 'Ethanol'] # SY added  ethanol
category2 = list(df.country.unique()[1:])
category2.sort() # 알파벳 정렬
category3 = list(df_PoA_final.PoA.unique()[0:])
results_category = category1 + category2 + category3

In [27]:
results_category

['All',
 'GS-Metered-Pellets',
 'GS-LPG',
 'GS-Firewood',
 'GS-Simplified Firewood',
 'GS-Charcoal',
 'CDM-Charcoal',
 'CDM_Ethanol',
 'CDM-Firewood',
 'Ethanol',
 'Bangladesh',
 'Burundi',
 'Cambodia',
 'Fiji',
 'Ghana',
 'Guatemala',
 'Kenya',
 'Lao',
 'Madagascar',
 'Malawi',
 'Myanmar',
 'Nigeria',
 'Uganda',
 'Viet Nam',
 'Zambia',
 5341,
 5342,
 6864,
 7359,
 7997,
 8480,
 9007,
 9265,
 9956,
 10008,
 10030,
 10415,
 10430,
 10431,
 10443,
 10471,
 10474,
 10476,
 10477,
 10497,
 10576]

In [28]:
results_lower = np.array([
    round(ci_lower, 2), 0, 0, 0, 0, 0,
                 round(df_combination_final['lower'][0],2),
                 round(df_combination_final['lower'][1],2),
                 round(df_combination_final['lower'][2],2),
                 round(df_combination_final['lower'][3],2),
                 round(df_country_final['lower'][0],2),
                 round(df_country_final['lower'][1],2),
                 round(df_country_final['lower'][2],2),
                 round(df_country_final['lower'][3],2),
                 round(df_country_final['lower'][4],2),
                 round(df_country_final['lower'][5],2),
                 round(df_country_final['lower'][6],2),
                 round(df_country_final['lower'][7],2),
                 round(df_country_final['lower'][8],2),
                 round(df_country_final['lower'][9],2),
                 round(df_country_final['lower'][10],2),
                 round(df_country_final['lower'][11],2),
                 round(df_country_final['lower'][12],2),
                 round(df_country_final['lower'][13],2),
                 round(df_country_final['lower'][14],2),
                 round(df_PoA_final['lower'][0],2),
                 round(df_PoA_final['lower'][1],2),
                 round(df_PoA_final['lower'][2],2),
                 round(df_PoA_final['lower'][3],2),
                 round(df_PoA_final['lower'][4],2),
                 round(df_PoA_final['lower'][5],2),
                 round(df_PoA_final['lower'][6],2),
                 round(df_PoA_final['lower'][7],2),
                 round(df_PoA_final['lower'][8],2),
                 round(df_PoA_final['lower'][9],2),
                 round(df_PoA_final['lower'][10],2),
                 round(df_PoA_final['lower'][11],2),
                 round(df_PoA_final['lower'][12],2),
                 round(df_PoA_final['lower'][13],2),
                 round(df_PoA_final['lower'][14],2),
                 round(df_PoA_final['lower'][15],2),
                 round(df_PoA_final['lower'][16],2),
                 round(df_PoA_final['lower'][17],2),
                 round(df_PoA_final['lower'][18],2),
                 round(df_PoA_final['lower'][19],2),
                 round(df_PoA_final['lower'][20],2)])

results_mean = np.array([
    round(reported_value, 2), 0, 0, 0, 0, 0,
                 round(df_combination_final['total'][0],2),
                 round(df_combination_final['total'][1],2),
                 round(df_combination_final['total'][2],2),
                 round(df_combination_final['total'][3],2),
                 round(df_country_final['total'][0],2),
                 round(df_country_final['total'][1],2),
                 round(df_country_final['total'][2],2),
                 round(df_country_final['total'][3],2),
                 round(df_country_final['total'][4],2),
                 round(df_country_final['total'][5],2),
                 round(df_country_final['total'][6],2),
                 round(df_country_final['total'][7],2),
                 round(df_country_final['total'][8],2),
                 round(df_country_final['total'][9],2),
                 round(df_country_final['total'][10],2),
                 round(df_country_final['total'][11],2),
                 round(df_country_final['total'][12],2),
                 round(df_country_final['total'][13],2),
                 round(df_country_final['total'][14],2),
                 round(df_PoA_final['total'][0],2),
                 round(df_PoA_final['total'][1],2),
                 round(df_PoA_final['total'][2],2),
                 round(df_PoA_final['total'][3],2),
                 round(df_PoA_final['total'][4],2),
                 round(df_PoA_final['total'][5],2),
                 round(df_PoA_final['total'][6],2),
                 round(df_PoA_final['total'][7],2),
                 round(df_PoA_final['total'][8],2),
                 round(df_PoA_final['total'][9],2),
                 round(df_PoA_final['total'][10],2),
                 round(df_PoA_final['total'][11],2),
                 round(df_PoA_final['total'][12],2),
                 round(df_PoA_final['total'][13],2),
                 round(df_PoA_final['total'][14],2),
                 round(df_PoA_final['total'][15],2),
                 round(df_PoA_final['total'][16],2),
                 round(df_PoA_final['total'][17],2),
                 round(df_PoA_final['total'][18],2),
                 round(df_PoA_final['total'][19],2),
                 round(df_PoA_final['total'][20],2)

])

results_high = np.array([
    round(ci_upper, 2), 0, 0, 0, 0, 0,
                 round(df_combination_final['upper'][0],2),
                 round(df_combination_final['upper'][1],2),
                 round(df_combination_final['upper'][2],2),
                 round(df_combination_final['upper'][3],2),
                 round(df_country_final['upper'][0],2),
                 round(df_country_final['upper'][1],2),
                 round(df_country_final['upper'][2],2),
                 round(df_country_final['upper'][3],2),
                 round(df_country_final['upper'][4],2),
                 round(df_country_final['upper'][5],2),
                 round(df_country_final['upper'][6],2),
                 round(df_country_final['upper'][7],2),
                 round(df_country_final['upper'][8],2),
                 round(df_country_final['upper'][9],2),
                 round(df_country_final['upper'][10],2),
                 round(df_country_final['upper'][11],2),
                 round(df_country_final['upper'][12],2),
                 round(df_country_final['upper'][13],2),
                 round(df_country_final['upper'][14],2),
                 round(df_PoA_final['upper'][0],2),
                 round(df_PoA_final['upper'][1],2),
                 round(df_PoA_final['upper'][2],2),
                 round(df_PoA_final['upper'][3],2),
                 round(df_PoA_final['upper'][4],2),
                 round(df_PoA_final['upper'][5],2),
                 round(df_PoA_final['upper'][6],2),
                 round(df_PoA_final['upper'][7],2),
                 round(df_PoA_final['upper'][8],2),
                 round(df_PoA_final['upper'][9],2),
                 round(df_PoA_final['upper'][10],2),
                 round(df_PoA_final['upper'][11],2),
                 round(df_PoA_final['upper'][12],2),
                 round(df_PoA_final['upper'][13],2),
                 round(df_PoA_final['upper'][14],2),
                 round(df_PoA_final['upper'][15],2),
                 round(df_PoA_final['upper'][16],2),
                 round(df_PoA_final['upper'][17],2),
                 round(df_PoA_final['upper'][18],2),
                 round(df_PoA_final['upper'][19],2),
                 round(df_PoA_final['upper'][20],2)])

In [29]:
# Create DataFrame

df_All_results_table = [results_lower,results_mean, results_high]
results_category = np.array(results_category)
x = pd.DataFrame(df_All_results_table)
x = x.transpose()
x['Category'] = results_category[:]
x
x.to_excel('df_usage_results_table.xlsx')

In [30]:
x

,0,1,2,Category
0,1.17,1.72,2.27,All
1,0.00,0.00,0.00,GS-Metered-Pellets
2,0.00,0.00,0.00,GS-LPG
3,0.00,0.00,0.00,GS-Firewood
4,0.00,0.00,0.00,GS-Simplified Firewood
5,0.00,0.00,0.00,GS-Charcoal
6,1.69,1.76,1.83,CDM-Charcoal
7,1.32,1.43,1.55,CDM_Ethanol
8,1.83,1.85,1.88,CDM-Firewood
9,1.90,1.96,2.01,Ethanol
